#### Aquí encontrarás todo lo que necesitas saber: https://www.kaggle.com/t/ab8726f0cfc84544abbae69a6be88071

## Librerias

In [1]:
import pandas as pd
import numpy as np
import re
import urllib.request
from PIL import Image

## Data

In [57]:
### Para que funcione necesitas bajarte los archivos de datos de Kaggle 
train = pd.read_csv("data/train.csv")


## Data exploration

In [54]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 912 entries, 0 to 911
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   laptop_ID         912 non-null    int64  
 1   Company           912 non-null    object 
 2   Product           912 non-null    object 
 3   TypeName          912 non-null    object 
 4   Inches            912 non-null    float64
 5   ScreenResolution  912 non-null    object 
 6   Cpu               912 non-null    object 
 7   Ram               912 non-null    object 
 8   Memory            912 non-null    object 
 9   Gpu               912 non-null    object 
 10  OpSys             912 non-null    object 
 11  Weight            912 non-null    object 
 12  Price_euros       912 non-null    float64
dtypes: float64(2), int64(1), object(10)
memory usage: 92.8+ KB


In [4]:
train.head()

,laptop_ID,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price_euros
0,1104,Acer,Aspire ES1-523,Notebook,15.6,1366x768,AMD A8-Series 7410 2.2GHz,4GB,500GB HDD,AMD Radeon R5,Windows 10,2.4kg,387.0
1,114,Dell,XPS 13,Ultrabook,13.3,Quad HD+ / Touchscreen 3200x1800,Intel Core i7 7560U 2.4GHz,8GB,256GB SSD,Intel Iris Plus Graphics 640,Windows 10,1.23kg,1379.0
2,172,Acer,Aspire A517-51G,Notebook,17.3,IPS Panel Full HD 1920x1080,Intel Core i5 8250U 1.6GHz,8GB,256GB SSD,Nvidia GeForce MX150,Windows 10,3kg,854.0
3,918,HP,Elitebook 820,Netbook,12.5,Full HD 1920x1080,Intel Core i7 7500U 2.7GHz,8GB,512GB SSD,Intel HD Graphics 620,Windows 10,1.26kg,1483.0
4,447,Lenovo,IdeaPad 320-15AST,Notebook,17.3,1600x900,AMD A6-Series 9220 2.5GHz,8GB,1TB HDD,AMD Radeon R4,Windows 10,2.8kg,519.0


## Data processing

In [137]:
# Hacemos una copia del df
train_copy = train.copy()

In [138]:
train_copy = train_copy[~train_copy.apply(lambda row: row.astype(str).str.contains('Netbook').any(), axis=1)]


In [139]:
# Vemos cuantos valores nulos tenemos 
train_copy[train_copy.isnull().any(axis=1)]

,laptop_ID,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price_euros


In [140]:
train_copy.drop(['TypeName','Product','Company'],axis=1, inplace=True)

In [62]:
train_copy.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 893 entries, 0 to 911
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   laptop_ID         893 non-null    int64  
 1   Inches            893 non-null    float64
 2   ScreenResolution  893 non-null    object 
 3   Cpu               893 non-null    object 
 4   Ram               893 non-null    object 
 5   Memory            893 non-null    object 
 6   Gpu               893 non-null    object 
 7   OpSys             893 non-null    object 
 8   Weight            893 non-null    object 
 9   Price_euros       893 non-null    float64
dtypes: float64(2), int64(1), object(7)
memory usage: 76.7+ KB


In [63]:
train_copy.head()

,laptop_ID,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price_euros
0,1104,15.6,1366x768,AMD A8-Series 7410 2.2GHz,4GB,500GB HDD,AMD Radeon R5,Windows 10,2.4kg,387.0
1,114,13.3,Quad HD+ / Touchscreen 3200x1800,Intel Core i7 7560U 2.4GHz,8GB,256GB SSD,Intel Iris Plus Graphics 640,Windows 10,1.23kg,1379.0
2,172,17.3,IPS Panel Full HD 1920x1080,Intel Core i5 8250U 1.6GHz,8GB,256GB SSD,Nvidia GeForce MX150,Windows 10,3kg,854.0
4,447,17.3,1600x900,AMD A6-Series 9220 2.5GHz,8GB,1TB HDD,AMD Radeon R4,Windows 10,2.8kg,519.0
5,1150,15.6,Full HD 1920x1080,Intel Core i7 6500U 2.5GHz,16GB,1.0TB Hybrid,AMD Radeon R7 M360,Windows 10,2.5kg,1099.0


In [141]:
def remove_str_and_convert_to_int(df, column_name,):
    df[column_name] = df[column_name].str.replace('\D+', '').astype(int)

In [142]:
# Eliminamos los str de la columna Memory
remove_str_and_convert_to_int(train_copy, 'Memory')

# Eliminamos los str de la columna Ram
remove_str_and_convert_to_int(train_copy,'Ram')

C:\Users\varel\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.7_qbz5n2kfra8p0\LocalCache\local-packages\Python37\site-packages\ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  


In [143]:
def remove_str_and_convert_to_float(df, column_name):
    df[column_name] = df[column_name].str.replace('\D+', '').astype(float)

In [144]:
#hacemos un ramplazo para normalizar los valores 

train_copy['Memory'] = train_copy['Memory'].replace({1: 1000, 10: 1000, 256500: 756, 1281: 1128, 2561: 1256, 
                                                       5121: 1512, 512256: 768,  5122: 2512, 
                                                       256256: 512, 11: 2000, 2: 2000})

In [145]:
# Eliminamos el str Kg de la columna Weight y normalizamos los valores 

train_copy['Weight'] = train_copy['Weight'].apply(lambda x: int(float(x.replace('kg', '').strip()) * 1000))

In [146]:
train_copy["OpSys"] = train_copy["OpSys"].replace("No OS", "macOS")

In [147]:
# cada sistema operativo se convertirá en un número entero en la columna "OpSys".

os_dict = {'Windows 10': 1,  'Linux': 2, 'Mac OS X': 3, 'Windows 7': 4, 'Chrome OS': 5, 'macOS': 6, 'Windows 10 S': 7}
train_copy['OpSys'] = train_copy['OpSys'].map(os_dict)

In [148]:
def extract_speed(text):
    speed = re.findall(r'\d+\.*\d*GHz', text)[0]
    speed = speed.replace('GHz', '')
    return float(speed)


In [149]:
train_copy['Cpu_GHz'] = train_copy['Cpu'].apply(extract_speed).astype(float)


In [150]:
def split_cpu(cpu):
    letters = ''.join(filter(str.isalpha, cpu))
    numbers = ''.join(filter(str.isdigit, cpu))
    return letters, numbers

In [151]:
train_copy[['Cpu_letters', 'Cpu_numbers']] = train_copy['Cpu'].apply(split_cpu).apply(pd.Series)


In [152]:
train_copy[['Gpu_letters', 'Gpu_numbers']] = train_copy['Gpu'].apply(split_cpu).apply(pd.Series)

In [153]:
train_copy.head()

,laptop_ID,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price_euros,Cpu_GHz,Cpu_letters,Cpu_numbers,Gpu_letters,Gpu_numbers
0,1104,15.6,1366x768,AMD A8-Series 7410 2.2GHz,4,500,AMD Radeon R5,1.0,2400,387.0,2.2,AMDASeriesGHz,8741022,AMDRadeonR,5
1,114,13.3,Quad HD+ / Touchscreen 3200x1800,Intel Core i7 7560U 2.4GHz,8,256,Intel Iris Plus Graphics 640,1.0,1230,1379.0,2.4,IntelCoreiUGHz,7756024,IntelIrisPlusGraphics,640
2,172,17.3,IPS Panel Full HD 1920x1080,Intel Core i5 8250U 1.6GHz,8,256,Nvidia GeForce MX150,1.0,3000,854.0,1.6,IntelCoreiUGHz,5825016,NvidiaGeForceMX,150
4,447,17.3,1600x900,AMD A6-Series 9220 2.5GHz,8,1000,AMD Radeon R4,1.0,2800,519.0,2.5,AMDASeriesGHz,6922025,AMDRadeonR,4
5,1150,15.6,Full HD 1920x1080,Intel Core i7 6500U 2.5GHz,16,1000,AMD Radeon R7 M360,1.0,2500,1099.0,2.5,IntelCoreiUGHz,7650025,AMDRadeonRM,7360


In [154]:
train_copy.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 893 entries, 0 to 911
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   laptop_ID         893 non-null    int64  
 1   Inches            893 non-null    float64
 2   ScreenResolution  893 non-null    object 
 3   Cpu               893 non-null    object 
 4   Ram               893 non-null    int32  
 5   Memory            893 non-null    int64  
 6   Gpu               893 non-null    object 
 7   OpSys             892 non-null    float64
 8   Weight            893 non-null    int64  
 9   Price_euros       893 non-null    float64
 10  Cpu_GHz           893 non-null    float64
 11  Cpu_letters       893 non-null    object 
 12  Cpu_numbers       893 non-null    object 
 13  Gpu_letters       893 non-null    object 
 14  Gpu_numbers       893 non-null    object 
dtypes: float64(4), int32(1), int64(3), object(7)
memory usage: 108.1+ KB


In [155]:
# Aplicar la función str.extract() para extraer los números
train_copy[['resolucion_x', 'resolucion_y']] = train_copy['ScreenResolution'].str.extract('(\d+)x(\d+)', expand=True)

# Imprimir el resultado
print(train_copy[['ScreenResolution', 'resolucion_x', 'resolucion_y']])


                     ScreenResolution resolucion_x resolucion_y
0                            1366x768         1366          768
1    Quad HD+ / Touchscreen 3200x1800         3200         1800
2         IPS Panel Full HD 1920x1080         1920         1080
4                            1600x900         1600          900
5                   Full HD 1920x1080         1920         1080
..                                ...          ...          ...
907                 Full HD 1920x1080         1920         1080
908             4K Ultra HD 3840x2160         3840         2160
909                 Full HD 1920x1080         1920         1080
910                 Full HD 1920x1080         1920         1080
911                 Full HD 1920x1080         1920         1080

[893 rows x 3 columns]


In [156]:
# Se crea una columna llamada total_pixels y se aplica la función lambda 
train_copy['total_pixels'] = train_copy.apply(lambda row: int(row['resolucion_x']) * int(row['resolucion_y']), axis=1)

In [157]:
# Se eliminan las columnas que no son necesarias 
train_copy.drop(['ScreenResolution','Cpu','Gpu','resolucion_x','resolucion_y'],axis=1, inplace=True)

In [158]:
train_copy.describe()

,laptop_ID,Inches,Ram,Memory,OpSys,Weight,Price_euros,Cpu_GHz,total_pixels
count,893.000000,893.00000,893.000000,893.000000,892.000000,893.000000,893.000000,893.000000,8.930000e+02
mean,666.284434,15.04972,8.340426,1181.727884,1.589686,2037.825308,1126.348690,2.309071,2.191541e+06
std,379.542827,1.35384,4.824325,17136.561483,1.463290,650.605078,680.396361,0.502688,1.436541e+06
min,1.000000,10.10000,2.000000,16.000000,1.000000,690.000000,199.000000,0.900000,1.049088e+06
25%,339.000000,14.00000,4.000000,256.000000,1.000000,1540.000000,615.000000,2.000000,1.440000e+06
50%,673.000000,15.60000,8.000000,500.000000,1.000000,2040.000000,985.000000,2.500000,2.073600e+06
75%,991.000000,15.60000,8.000000,1000.000000,1.000000,2300.000000,1477.000000,2.700000,2.073600e+06
max,1318.000000,18.40000,32.000000,512512.000000,7.000000,4600.000000,5499.000000,3.600000,8.294400e+06


In [160]:
train_copy.head()

,laptop_ID,Inches,Ram,Memory,OpSys,Weight,Price_euros,Cpu_GHz,Cpu_letters,Cpu_numbers,Gpu_letters,Gpu_numbers,total_pixels,Gpu_numbers\t
0,1104,15.6,4,500,1.0,2400,387.0,2.2,AMDASeriesGHz,8741022,AMDRadeonR,5,1049088,5
1,114,13.3,8,256,1.0,1230,1379.0,2.4,IntelCoreiUGHz,7756024,IntelIrisPlusGraphics,640,5760000,640
2,172,17.3,8,256,1.0,3000,854.0,1.6,IntelCoreiUGHz,5825016,NvidiaGeForceMX,150,2073600,150
4,447,17.3,8,1000,1.0,2800,519.0,2.5,AMDASeriesGHz,6922025,AMDRadeonR,4,1440000,4
5,1150,15.6,16,1000,1.0,2500,1099.0,2.5,IntelCoreiUGHz,7650025,AMDRadeonRM,7360,2073600,7360


In [159]:
train_copy['Gpu_numbers	'] = train_copy['Gpu_numbers'].str.strip()


In [161]:
print(train_copy['Gpu_numbers\t'].unique())

['5' '640' '150' '4' '7360' '620' '940' '1050' '520' '400' '515' '1070'
 '5330' '930' '920' '1060' '7445' '6000' '2' '615' '540' '950' '130' '505'
 '960' '' '500' '630' '1200' '5420' '970' '980' '530' '405' '580' '5300'
 '965' '5430' '5520' '510' '960039' '7440' '1080' '2200' '650' '4190'
 '1000' '3000' '17170' '7' '550' '9385' '5315' '5130' '7465' '7365' '7460'
 '455' '2000' '555']


In [162]:
train_copy.drop(['Gpu_letters','Cpu_letters'],axis=1, inplace=True)

In [163]:
train_copy['Cpu_numbers'] = train_copy['Cpu_numbers'].astype(float)


In [164]:
train_copy['Gpu_numbers\t'] = pd.to_numeric(train_copy['Gpu_numbers\t']).astype(float)


In [166]:
train_copy['Gpu_numbers\t'] = train_copy['Gpu_numbers\t'].fillna(train_copy['Gpu_numbers\t'].mean())


In [167]:
train_copy['OpSys'] = train_copy['OpSys'].fillna(train_copy['OpSys'].mean())


In [168]:
train_copy.isnull().sum()

laptop_ID        0
Inches           0
Ram              0
Memory           0
OpSys            0
Weight           0
Price_euros      0
Cpu_GHz          0
Cpu_numbers      0
Gpu_numbers      0
total_pixels     0
Gpu_numbers\t    0
dtype: int64

In [45]:
# train_copy['Gpu_numbers'] = pd.to_numeric(train_copy['Gpu_numbers'], errors='coerce')


In [46]:
# train_copy['Cpu_numbers'] = pd.to_numeric(train_copy['Cpu_numbers'], errors='coerce')


In [169]:
train_copy.head()


,laptop_ID,Inches,Ram,Memory,OpSys,Weight,Price_euros,Cpu_GHz,Cpu_numbers,Gpu_numbers,total_pixels,Gpu_numbers\t
0,1104,15.6,4,500,1.0,2400,387.0,2.2,8741022.0,5,1049088,5.0
1,114,13.3,8,256,1.0,1230,1379.0,2.4,7756024.0,640,5760000,640.0
2,172,17.3,8,256,1.0,3000,854.0,1.6,5825016.0,150,2073600,150.0
4,447,17.3,8,1000,1.0,2800,519.0,2.5,6922025.0,4,1440000,4.0
5,1150,15.6,16,1000,1.0,2500,1099.0,2.5,7650025.0,7360,2073600,7360.0


In [170]:
train_copy.drop(['Gpu_numbers'],axis=1, inplace=True)

In [171]:
train_copy.value_counts().sort_values(ascending=False)

laptop_ID  Inches  Ram  Memory  OpSys  Weight  Price_euros  Cpu_GHz  Cpu_numbers  total_pixels  Gpu_numbers\t
1          13.3    8    128     6.0    1370    1339.69      2.3      523.0        4096000       640.000000       1
26         15.6    4    1000    1.0    2300    418.64       2.0      360062.0     1049088       520.000000       1
4          15.4    16   512     6.0    1830    2537.45      2.7      727.0        5184000       455.000000       1
6          15.6    4    500     1.0    2100    400.00       3.0      994203.0     1049088       5.000000         1
7          15.4    16   256     3.0    2040    2139.97      2.2      722.0        5184000       3275.266361      1
                                                                                                                ..
1311       15.6    8    1000    1.0    2600    899.00       2.6      7670026.0    2073600       960.000000       1
1312       15.6    6    1000    1.0    2040    549.99       2.9      9941029.0    207

In [172]:
train_copy.describe()

,laptop_ID,Inches,Ram,Memory,OpSys,Weight,Price_euros,Cpu_GHz,Cpu_numbers,total_pixels,Gpu_numbers\t
count,893.000000,893.00000,893.000000,893.000000,893.000000,893.000000,893.000000,893.000000,8.930000e+02,8.930000e+02,893.000000
mean,666.284434,15.04972,8.340426,1181.727884,1.589686,2037.825308,1126.348690,2.309071,6.107455e+06,2.191541e+06,3275.266361
std,379.542827,1.35384,4.824325,17136.561483,1.462470,650.605078,680.396361,0.502688,6.458321e+06,1.436541e+06,45381.024817
min,1.000000,10.10000,2.000000,16.000000,1.000000,690.000000,199.000000,0.900000,1.100000e+01,1.049088e+06,2.000000
25%,339.000000,14.00000,4.000000,256.000000,1.000000,1540.000000,615.000000,2.000000,3.713027e+06,1.440000e+06,520.000000
50%,673.000000,15.60000,8.000000,500.000000,1.000000,2040.000000,985.000000,2.500000,5.730026e+06,2.073600e+06,620.000000
75%,991.000000,15.60000,8.000000,1000.000000,1.000000,2300.000000,1477.000000,2.700000,7.750027e+06,2.073600e+06,1000.000000
max,1318.000000,18.40000,32.000000,512512.000000,7.000000,4600.000000,5499.000000,3.600000,7.650025e+07,8.294400e+06,960039.000000


In [57]:
train_copy.dropna(inplace=True)


In [103]:
train_copy['Gpu_numbers'].isnull().value_counts()

False    893
Name: Gpu_numbers, dtype: int64

In [173]:
train_copy.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 893 entries, 0 to 911
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   laptop_ID     893 non-null    int64  
 1   Inches        893 non-null    float64
 2   Ram           893 non-null    int32  
 3   Memory        893 non-null    int64  
 4   OpSys         893 non-null    float64
 5   Weight        893 non-null    int64  
 6   Price_euros   893 non-null    float64
 7   Cpu_GHz       893 non-null    float64
 8   Cpu_numbers   893 non-null    float64
 9   total_pixels  893 non-null    int64  
 10  Gpu_numbers	  893 non-null    float64
dtypes: float64(6), int32(1), int64(4)
memory usage: 80.2 KB


-----------------------------------------------------------------------------------------------------------------

## Pasos que si o si debéis realizar para poder participar en la competición:

### 1. Definir X e y

In [174]:
X = train_copy.drop(['Price_euros'], axis=1)
y = train_copy['Price_euros'].copy()
X.shape

(893, 10)

In [175]:
y.shape

(893,)

In [176]:
y.head

<bound method NDFrame.head of 0       387.00
1      1379.00
2       854.00
4       519.00
5      1099.00
        ...   
907     549.00
908    3659.40
909    1549.00
910    2729.00
911     478.89
Name: Price_euros, Length: 893, dtype: float64>

In [177]:
X.head()

,laptop_ID,Inches,Ram,Memory,OpSys,Weight,Cpu_GHz,Cpu_numbers,total_pixels,Gpu_numbers\t
0,1104,15.6,4,500,1.0,2400,2.2,8741022.0,1049088,5.0
1,114,13.3,8,256,1.0,1230,2.4,7756024.0,5760000,640.0
2,172,17.3,8,256,1.0,3000,1.6,5825016.0,2073600,150.0
4,447,17.3,8,1000,1.0,2800,2.5,6922025.0,1440000,4.0
5,1150,15.6,16,1000,1.0,2500,2.5,7650025.0,2073600,7360.0


In [178]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 893 entries, 0 to 911
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   laptop_ID     893 non-null    int64  
 1   Inches        893 non-null    float64
 2   Ram           893 non-null    int32  
 3   Memory        893 non-null    int64  
 4   OpSys         893 non-null    float64
 5   Weight        893 non-null    int64  
 6   Cpu_GHz       893 non-null    float64
 7   Cpu_numbers   893 non-null    float64
 8   total_pixels  893 non-null    int64  
 9   Gpu_numbers	  893 non-null    float64
dtypes: float64(5), int32(1), int64(4)
memory usage: 73.3 KB


### 2. Dividir X_train, X_test, y_train, y_test

In [110]:
from sklearn.model_selection import train_test_split

In [179]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 42)

In [180]:
X_train

,laptop_ID,Inches,Ram,Memory,OpSys,Weight,Cpu_GHz,Cpu_numbers,total_pixels,Gpu_numbers\t
338,1018,13.3,4,128,1.0,1200,2.3,5620023.0,2073600,520.0
747,179,15.6,8,1000,1.0,2300,2.4,3710024.0,1049088,620.0
391,1232,17.3,16,1256,1.0,2900,2.8,7770028.0,2073600,1060.0
717,593,17.3,16,1256,1.0,3780,2.8,7770028.0,2073600,1050.0
827,253,13.3,4,500,1.0,1490,1.6,5825016.0,2073600,620.0
...,...,...,...,...,...,...,...,...,...,...
108,554,15.6,4,1000,6.0,2150,2.3,5620023.0,2073600,520.0
275,950,15.6,4,1000,1.0,2180,2.0,3600620.0,1049088,520.0
878,1040,14.0,4,500,1.0,2310,2.5,5720025.0,1049088,620.0
445,82,15.6,8,256,1.0,1880,1.6,5825016.0,2073600,620.0


### 3. Asignar el modelo (vacío) a una variable


In [113]:
from sklearn.linear_model import LinearRegression

modelo = LinearRegression()

In [181]:
modelo.fit(X_train, y_train)

LinearRegression()

In [182]:
predicciones = modelo.predict(X_test)

In [183]:
predicciones

array([ 662.24814505,  848.98390604, 1217.71578084,  687.14288334,
       1963.79604141, 1275.86519961,  213.74500927,  650.31985123,
       2423.3590235 ,  389.73446749, 1305.37588831, 1154.7639413 ,
       1232.83494298,  299.01732776,  728.42098621, 3102.13996521,
       1143.2099192 , 2422.07355992, 1297.9375066 , 1518.09116885,
       1254.14732305, 1248.04261008,  589.33028747,  857.3068091 ,
       1778.44634159, 1161.44648712,  720.18830362, 1177.09964982,
       1308.90460794,  744.67059151, 1235.59537818,  391.01993107,
       1276.4839859 , 1202.26032631, 2463.31665522, 1749.74613226,
        467.82919387,  719.32832074,  383.00441855, 1489.1805473 ,
        265.33209073,  989.37605541, 1154.83449858, 1230.57182872,
        512.60985982, 1247.20530974, 1206.24710627, 1695.15307945,
        973.74120915,  690.65997177, 1402.38488203,  647.27878674,
        795.05373764,  713.68850456, 1611.42120366,  841.1220386 ,
        718.47255027,  844.17849895,  474.75373013,  458.25225

### 4. Sacar métricas, valorar el modelo 

Recuerda que en la competición se va a evaluar con la métrica de MAE. 

In [184]:
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

print('RMSE',mean_squared_error(y_test, predicciones))

RMSE 168981.22515934194


-----------------------------------------------------------------

## Una vez listo el modelo, toca predecir con el dataset de predicción 

Definición de **modelo que está listo**. 

Tras hacer suficientes pruebas, analizar los datos, hacer feature engineering, probar diferentes modelos con diferentes parámetros, es con este con el que observo mejores métricas y menos overfitting. ¡Cuidado con el overfitting aquí! Si vuestro modelo aprende muy bien de estos datos pero hay overfitting cuando le pasemos los datos desconocidos de `test.csv` nos arriesgamos a que digamos, no salga lo esperado.

### 1. Entrena dicho modelo con TODOS tus datos de train, esto es con `train.csv` al completo.


**CON LAS TRANSFORMACIONES QUE LE HAYAS REALIZADO A `X` INCLUÍDAS.**


Véase:
- Estandarización/Normalización
- Eliminación de Outliers
- Eliminación de columnas
- Creación de columnas nuevas
- Gestión de valores nulos
- Y un largo etcétera de técnicas que como Data Scientist hayas considerado las mejores para tu dataset.

### 2. Carga los datos de `test.csv` para predecir.

In [185]:
test_pred = pd.read_csv("data/test.csv")
test_pred.head()

,laptop_ID,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight
0,750,Toshiba,Tecra X40-D-10G,Notebook,14.0,IPS Panel Full HD / Touchscreen 1920x1080,Intel Core i5 7200U 2.5GHz,8GB,256GB SSD,Intel HD Graphics 620,Windows 10,1.25kg
1,726,Mediacom,Smartbook 142,Notebook,14.0,IPS Panel Full HD 1920x1200,Intel Atom Z8350 1.92GHz,4GB,32GB HDD,Intel HD Graphics 400,Windows 10,1.4kg
2,633,Mediacom,SmartBook 140,Notebook,14.0,Full HD 1920x1080,Intel Atom x5-Z8350 1.44GHz,2GB,32GB Flash Storage,Intel HD Graphics,Windows 10,1.4kg
3,363,Dell,Inspiron 7577,Gaming,15.6,Full HD 1920x1080,Intel Core i5 7300HQ 2.5GHz,8GB,1TB HDD,Nvidia GeForce GTX 1050,Windows 10,2.65kg
4,319,Asus,VivoBook Flip,2 in 1 Convertible,11.6,Touchscreen 1366x768,Intel Celeron Dual Core N3350 1.1GHz,2GB,32GB Flash Storage,Intel HD Graphics 500,Windows 10,1.1kg


In [186]:
test_pred_copy = test_pred.copy()

In [187]:
test_pred_copy[test_pred_copy.isnull().any(axis=1)]

,laptop_ID,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight


In [188]:
test_pred_copy.drop(['TypeName','Product','Company'],axis=1, inplace=True)

In [189]:
test_pred_copy.dropna(axis=0, inplace=True)

In [190]:
def remove_str_and_convert_to_int(df, column_name,):
    df[column_name] = df[column_name].str.replace('\D+', '').astype(int)

In [191]:
# Eliminamos los str de la columna Memory
remove_str_and_convert_to_int(test_pred_copy, 'Memory')

# Eliminamos los str de la columna Ram
remove_str_and_convert_to_int(test_pred_copy,'Ram')

C:\Users\varel\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.7_qbz5n2kfra8p0\LocalCache\local-packages\Python37\site-packages\ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  


In [192]:
def remove_str_and_convert_to_float(df, column_name):
    df[column_name] = df[column_name].str.replace('\D+', '').astype(float)

In [193]:
#hacemos un ramplazo para normalizar los valores 

test_pred_copy['Memory'] = test_pred_copy['Memory'].replace({1: 1000, 10: 1000, 256500: 756, 1281: 1128, 2561: 1256, 
                                                       5121: 1512, 512256: 768,  5122: 2512, 
                                                       256256: 512, 11: 2000, 2: 2000})

In [194]:
# Eliminamos el str Kg de la columna Weight y normalizamos los valores 

test_pred_copy['Weight'] = test_pred_copy['Weight'].apply(lambda x: int(float(x.replace('kg', '').strip()) * 1000))

In [195]:
test_pred_copy["OpSys"] = test_pred_copy["OpSys"].replace("No OS", "macOS")

In [196]:
# cada sistema operativo se convertirá en un número entero en la columna "OpSys".

os_dict = {'Windows 10': 1,  'Linux': 2, 'Mac OS X': 3, 'Windows 7': 4, 'Chrome OS': 5, 'macOS': 6, 'Windows 10 S': 7}
test_pred_copy['OpSys'] = test_pred_copy['OpSys'].map(os_dict)

In [197]:
def extract_speed(text):
    speed = re.findall(r'\d+\.*\d*GHz', text)[0]
    speed = speed.replace('GHz', '')
    return float(speed)

In [198]:
test_pred_copy['Cpu_GHz'] = test_pred_copy['Cpu'].apply(extract_speed).astype(float)

In [199]:
def split_cpu(cpu):
    letters = ''.join(filter(str.isalpha, cpu))
    numbers = ''.join(filter(str.isdigit, cpu))
    return letters, numbers


In [200]:
test_pred_copy[['Cpu_letters', 'Cpu_numbers']] = test_pred_copy['Cpu'].apply(split_cpu).apply(pd.Series)


In [201]:
test_pred_copy[['Gpu_letters', 'Gpu_numbers']] = test_pred_copy['Gpu'].apply(split_cpu).apply(pd.Series)

In [202]:


# Aplicar la función str.extract() para extraer los números
test_pred_copy[['resolucion_x', 'resolucion_y']] = test_pred_copy['ScreenResolution'].str.extract('(\d+)x(\d+)', expand=True)

# Imprimir el resultado
print(test_pred_copy[['ScreenResolution', 'resolucion_x', 'resolucion_y']])


                              ScreenResolution resolucion_x resolucion_y
0    IPS Panel Full HD / Touchscreen 1920x1080         1920         1080
1                  IPS Panel Full HD 1920x1200         1920         1200
2                            Full HD 1920x1080         1920         1080
3                            Full HD 1920x1080         1920         1080
4                         Touchscreen 1366x768         1366          768
..                                         ...          ...          ...
386                          Full HD 1920x1080         1920         1080
387        Touchscreen / 4K Ultra HD 3840x2160         3840         2160
388                          Full HD 1920x1080         1920         1080
389                                   1366x768         1366          768
390        4K Ultra HD / Touchscreen 3840x2160         3840         2160

[391 rows x 3 columns]


In [203]:
test_pred_copy['total_pixels'] = test_pred_copy.apply(lambda row: int(row['resolucion_x']) * int(row['resolucion_y']), axis=1)


In [204]:
# Se eliminan las columnas que no son necesarias 
test_pred_copy.drop(['ScreenResolution','Cpu','Gpu','resolucion_x','resolucion_y'],axis=1, inplace=True)

In [205]:
test_pred_copy.drop(['Gpu_letters','Cpu_letters'],axis=1, inplace=True)

In [211]:
test_pred_copy['Gpu_numbers	'] = test_pred_copy['Gpu_numbers'].str.strip()


In [212]:
test_pred_copy['Cpu_numbers'] = test_pred_copy['Cpu_numbers'].astype(float)


In [213]:
test_pred_copy['Gpu_numbers\t'] = pd.to_numeric(test_pred_copy['Gpu_numbers\t']).astype(float)


In [214]:
test_pred_copy['Gpu_numbers\t'] = test_pred_copy['Gpu_numbers\t'].fillna(test_pred_copy['Gpu_numbers\t'].mean())


In [215]:
test_pred_copy['OpSys'] = test_pred_copy['OpSys'].fillna(test_pred_copy['OpSys'].mean())


In [217]:
test_pred_copy.drop(['Gpu_numbers'],axis=1, inplace=True)

In [134]:
# test_pred_copy['Gpu_numbers'] = pd.to_numeric(test_pred_copy['Gpu_numbers'], errors='coerce')


In [135]:
# test_pred_copy['Cpu_numbers'] = pd.to_numeric(test_pred_copy['Cpu_numbers'], errors='coerce')


In [103]:
# test_pred_copy.dropna(inplace=True)


In [208]:
test_pred_copy.fillna(value=0, inplace=True)

In [218]:
test_pred_copy.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 391 entries, 0 to 390
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   laptop_ID     391 non-null    int64  
 1   Inches        391 non-null    float64
 2   Ram           391 non-null    int32  
 3   Memory        391 non-null    int64  
 4   OpSys         391 non-null    float64
 5   Weight        391 non-null    int64  
 6   Cpu_GHz       391 non-null    float64
 7   Cpu_numbers   391 non-null    float64
 8   total_pixels  391 non-null    int64  
 9   Gpu_numbers	  391 non-null    float64
dtypes: float64(5), int32(1), int64(4)
memory usage: 32.1 KB


In [219]:
predictions_submit = modelo.predict(test_pred_copy)
predictions_submit

array([1184.6662247 ,  749.63436353,  460.86844627, 1122.94913441,
        342.10140229,  590.75344232,  435.3371096 , 1831.86463752,
       1735.6001092 , 1254.54605562,  994.15734214,  974.25356861,
       1110.41589939,  936.0587005 ,  572.93775015, 1063.93486142,
        934.91955911, 1951.44146341,  533.0607562 , 1168.27052681,
       1180.5264641 ,  991.86291687,  581.22464698, 1004.86774688,
        365.88796207, 1391.02900384,  878.72358532,  754.84676021,
       1608.12230311, 1233.24892093,  851.7515829 , 1140.77147284,
       1000.4441345 , 1171.10543682, 1180.79732988, 1206.25821911,
       1162.07698643, 1445.18011476,  837.03083637, 1886.44813281,
       1809.41366367, 1079.33213234,  493.42683502, 1588.32382286,
        796.39510438, 1888.97370043, 1216.2984733 , 1733.32604139,
       1031.56055824, 1753.2508967 ,  771.78882485,  617.33245666,
       1078.23371095, 1058.7583788 ,  430.85430156,  804.13348991,
       1073.45619412,  688.17285333, 1278.85016689, 1263.36317

IMPORTANTE: APLICAR LO MISMO A ESTOS DATOS QUE HAYÁIS APLICADO A LOS DATOS DE ENTRENAMIENTO

- SI EL ARRAY CON EL QUE HICISTEIS `.fit()` ERA DE 4 COLUMNAS, PARA `.predict()` DEBEN SER LAS MISMAS
- SI AL ARRAY CON EL QUE HICISTEIS `.fit()` LO NORMALIZASTEIS, PARA `.predict()` DEBÉIS NORMALIZARLO
- TODO IGUAL SALVO BORRAR FILAS, EL NÚMERO DE ROWS SE DEBE MANTENER EN ESTE SET, PUES LA PREDICCIÓN DEBE TENER 391 FILAS, SI O SI

**Entonces, si al cargar los datos de train usé `index_col=0` para que utilizara la primera columna del conjunto de datos como índice, ¿tendré que hacerlo también para el conjunto `test.csv`?**

In [ ]:
# ¿Qué opináis?
# Sí

![](https://www.tierraljelechu.com/web/img/wiki_up/1.996-SorpresaDto.-1-Red.jpg)

### 3. AHORA puedo hacer la predicción que será lo que subirás a Kaggle. 

**¿Qué es lo que subirás a Kaggle?**

In [220]:
predictions_submit = modelo.predict(test_pred_copy)
predictions_submit

array([1184.6662247 ,  749.63436353,  460.86844627, 1122.94913441,
        342.10140229,  590.75344232,  435.3371096 , 1831.86463752,
       1735.6001092 , 1254.54605562,  994.15734214,  974.25356861,
       1110.41589939,  936.0587005 ,  572.93775015, 1063.93486142,
        934.91955911, 1951.44146341,  533.0607562 , 1168.27052681,
       1180.5264641 ,  991.86291687,  581.22464698, 1004.86774688,
        365.88796207, 1391.02900384,  878.72358532,  754.84676021,
       1608.12230311, 1233.24892093,  851.7515829 , 1140.77147284,
       1000.4441345 , 1171.10543682, 1180.79732988, 1206.25821911,
       1162.07698643, 1445.18011476,  837.03083637, 1886.44813281,
       1809.41366367, 1079.33213234,  493.42683502, 1588.32382286,
        796.39510438, 1888.97370043, 1216.2984733 , 1733.32604139,
       1031.56055824, 1753.2508967 ,  771.78882485,  617.33245666,
       1078.23371095, 1058.7583788 ,  430.85430156,  804.13348991,
       1073.45619412,  688.17285333, 1278.85016689, 1263.36317

**¡PERO! Para subir a Kaggle la predicción, ésta tendrá que tener una forma específica y no valdrá otra.**

En este caso, la **MISMA** forma que `sample_submission.csv`. 

In [221]:
sample = pd.read_csv("data/sample_submission.csv")

In [222]:
sample.head()

,laptop_ID,Price_euros
0,750,500
1,726,500
2,633,500
3,363,650
4,319,650


In [223]:
sample.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 391 entries, 0 to 390
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype
---  ------       --------------  -----
 0   laptop_ID    391 non-null    int64
 1   Price_euros  391 non-null    int64
dtypes: int64(2)
memory usage: 6.2 KB


In [224]:
sample.shape

(391, 2)

### 4. Mete tus predicciones en un dataframe. 

En este caso, la **MISMA** forma que `sample_submission.csv`. 

In [225]:
submission = pd.DataFrame({"laptop_ID": sample['laptop_ID'], "Price_euros": predictions_submit})

In [226]:
submission.head()

,laptop_ID,Price_euros
0,750,1184.666225
1,726,749.634364
2,633,460.868446
3,363,1122.949134
4,319,342.101402


In [227]:
submission.shape

(391, 2)

### 5. Pásale el CHEQUEATOR para comprobar que efectivamente está listo para subir a Kaggle.

In [228]:
def chequeator(df_to_submit):
    """
    Esta función se asegura de que tu submission tenga la forma requerida por Kaggle.
    
    Si es así, se guardará el dataframe en un `csv` y estará listo para subir a Kaggle.
    
    Si no, LEE EL MENSAJE Y HAZLE CASO.
    
    Si aún no:
    - apaga tu ordenador, 
    - date una vuelta, 
    - enciendelo otra vez, 
    - abre este notebook y 
    - leelo todo de nuevo. 
    Todos nos merecemos una segunda oportunidad. También tú.
    """
    if df_to_submit.shape == sample.shape:
        if df_to_submit.columns.all() == sample.columns.all():
            if df_to_submit.laptop_ID.all() == sample.laptop_ID.all():
                print("You're ready to submit!")
                submission.to_csv("submission.csv", index = False) #muy importante el index = False
                urllib.request.urlretrieve("https://i.kym-cdn.com/photos/images/facebook/000/747/556/27a.jpg", "gfg.png")     
                img = Image.open("gfg.png")
                img.show()   
            else:
                print("Check the ids and try again")
        else:
            print("Check the names of the columns and try again")
    else:
        print("Check the number of rows and/or columns and try again")
        print("\nMensaje secreto de Clara: No me puedo creer que después de todo este notebook hayas hecho algún cambio en las filas de `diamonds_test.csv`. Lloro.")


In [229]:
chequeator(submission)

You're ready to submit!
